## XML输出解析器

该输出解析器允许用户以流行的 XML 格式从 LLM 获取结果。

请记住，大型语言模型是有漏洞的抽象！您必须使用具有足够容量的 LLM 来生成格式正确的 XML。


In [3]:
from langchain.output_parsers import XMLOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

api_key = "xxx"
base_url = "http://localhost:1234/v1"

chat = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3)


In [5]:
actor_quary = "生成李连杰的电影作品表"
output = chat.invoke(
f"""{actor_quary}
响应以xml的结构返回，使用如下xml结构
```
<xml>
<movie>电影1</movie>
<movie>电影2</movie>
<xml>
``` 
"""                   
)
print(output.content)


以下是一个基于李连杰部分电影作品列表的XML格式示例：

```xml
<xml>
    <movie>少林寺</movie>
    <movie>黄飞鸿系列</movie>
    <movie>方世玉</movie>
    <movie>致命罗宾逊</movie>
    <movie>致命武器4</movie>
    <movie>敢死队</movie>
    <movie>英雄</movie>
    <movie>霍元甲</movie>
    <movie>投名状</movie>
    <movie>功夫之王</movie>
</xml>
```

请注意，这个列表仅包含了一些李连杰的电影作品，并不全面。李连杰参与了多部电影，具体的作品表会更长且详细。


In [6]:
parser = XMLOutputParser()
parser.invoke(output)

{'xml': [{'movie': '少林寺'},
  {'movie': '黄飞鸿系列'},
  {'movie': '方世玉'},
  {'movie': '致命罗宾逊'},
  {'movie': '致命武器4'},
  {'movie': '敢死队'},
  {'movie': '英雄'},
  {'movie': '霍元甲'},
  {'movie': '投名状'},
  {'movie': '功夫之王'}]}

In [7]:
parser = XMLOutputParser()

format_instructions = """ 响应以xml的结构返回，使用如下xml结构
```
<xml>
<movie>电影1</movie>
<movie>电影2</movie>
<xml>
```
"""

prompt = PromptTemplate(
    template = """{quary}\n{format_instructions}""",
    input_variables = ["quary"],
    partial_variables = {"format_instructions": format_instructions}
)

chain = prompt | chat | parser

for chunk in chain.stream({"quary": actor_quary}):
    print(chunk, end="", flush=True)


{'xml': [{'movie': '少林寺'}]}{'xml': [{'movie': '黄飞鸿系列'}]}{'xml': [{'movie': '方世玉'}]}{'xml': [{'movie': '致命罗宾逊'}]}{'xml': [{'movie': '致命武器4'}]}{'xml': [{'movie': '英雄'}]}{'xml': [{'movie': '霍元甲'}]}{'xml': [{'movie': '投名状'}]}{'xml': [{'movie': '敢死队2'}]}{'xml': [{'movie': '龙门飞甲'}]}

In [8]:
parser = XMLOutputParser()

format_instructions = parser.get_format_instructions()
format_instructions

'The output should be formatted as a XML file.\n1. Output should conform to the tags below.\n2. If tags are not given, make them on your own.\n3. Remember to always open and close all the tags.\n\nAs an example, for the tags ["foo", "bar", "baz"]:\n1. String "<foo>\n   <bar>\n      <baz></baz>\n   </bar>\n</foo>" is a well-formatted instance of the schema.\n2. String "<foo>\n   <bar>\n   </foo>" is a badly-formatted instance.\n3. String "<foo>\n   <tag>\n   </tag>\n</foo>" is a badly-formatted instance.\n\nHere are the output tags:\n```\nNone\n```'